<a href="https://colab.research.google.com/github/AndrewRucker/Fantasy-Football-and-Betting/blob/main/BestBall/Underdog/BBM3/BBM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Library imports and dataset creation

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import ipywidgets as widgets
from IPython.core.pylabtools import figsize
from IPython.display import display

sns.set_context(font_scale=2)

# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

In [3]:
#Read in CSV to dataframe
url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/Rookie-and-Sophomores/RookSophTeams.csv'
df = pd.read_csv(url, index_col='Team')
df = df.dropna(how='all')

df

,POS,Name,TEAM,Round,Pick,ADP,Value,Draft Start,Cumulative Val
Team,,,,,,,,,
1.0,W/T,Jamar Chase,CIN,1.0,1.0,1.1,-0.1,4/13/2022,5.4
1.0,RB,Breece Hall,NaN,2.0,8.0,8.2,-0.2,4/13/2022,5.4
1.0,QB,Justin Fields,CHI,3.0,9.0,7.2,1.8,4/13/2022,5.4
1.0,QB,Mac Jones,NE,4.0,16.0,14.3,1.7,4/13/2022,5.4
1.0,RB,Elijah Mitchell,SF,5.0,17.0,17.5,-0.5,4/13/2022,5.4
...,...,...,...,...,...,...,...,...,...
150.0,W/T,Rashod Bateman,BAL,8.0,29.0,25.2,3.8,4/25/2022,9.4
150.0,RB,Kenneth Gainwell,PHI,9.0,36.0,37.4,-1.4,4/25/2022,9.4
150.0,RB,Trey Sermon,SF,10.0,37.0,37.0,0.0,4/25/2022,9.4


In [4]:
#Creation of dataset with player drafted with value according to ADP by draft date
some_values = ('Kenny Pickett', 'Matt Corral', 'Malik Willis', 'Sam Howell', 'Desmond Ridder', 'Trevor Lawrence', 'Justin Fields', 'Davis Mills', 'Mac Jones', 'Zach Wilson')
df_date = df.drop(columns=['POS', 'Round', 'Pick', 'ADP', 'Cumulative Val'])
df_date = df_date.loc[df_date['Name'].isin(some_values)]
df_date


,Name,TEAM,Value,Draft Start
Team,,,,
1.0,Justin Fields,CHI,1.8,4/13/2022
1.0,Mac Jones,NE,1.7,4/13/2022
2.0,Trevor Lawrence,JAX,0.1,4/13/2022
2.0,Zach Wilson,NYJ,2.5,4/13/2022
3.0,Zach Wilson,NYJ,-2.5,4/13/2022
...,...,...,...,...
147.0,Desmond Ridder,NaN,0.0,4/25/2022
148.0,Justin Fields,CHI,-2.6,4/25/2022
148.0,Malik Willis,NaN,0.3,4/25/2022


#### Initial data manipulation

In [5]:
df1 = df.groupby('Team')['POS'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

df1['RB'] = df1['POS'].str.count('RB')
df1['W/T'] = df1['POS'].str.count('W/T')
df1['QB'] = df1['POS'].str.count('QB')

df1['build'] = df1['QB'].astype(str) + "/" + df1['RB'].astype(str) + "/" + df1['W/T'].astype(str)
df1 = df1.drop(columns=['POS', 'RB', 'W/T', 'QB'])

In [6]:
df2 = pd.merge(df, df1, how='left', on = 'Team')
df3 = df2.groupby('Team')['Name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df2 = pd.merge(df2, df3, how='left', on='Team')

In [7]:
final_df = df2.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value'])
final_df = final_df.drop_duplicates()
final_df ['Players'] = final_df['Name_y']
final_df = final_df.drop(columns=['Name_y'])
final_df

split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12, 12 : 13, 13 : 14, 14 : 15, 15 : 16, 16 : 17, 17 : 18, 18 : 19, 19 : 20}, axis='columns', inplace=True)

In [8]:
df1 = pd.merge(df1, df2, how='left', on = 'Team')
df1 = df1.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value', 'Name_y'])
df1

,Team,build_x,Draft Start,Cumulative Val,build_y
0,1.0,2/4/6,4/13/2022,5.4,2/4/6
1,1.0,2/4/6,4/13/2022,5.4,2/4/6
2,1.0,2/4/6,4/13/2022,5.4,2/4/6
3,1.0,2/4/6,4/13/2022,5.4,2/4/6
4,1.0,2/4/6,4/13/2022,5.4,2/4/6
...,...,...,...,...,...
1795,150.0,2/4/6,4/25/2022,9.4,2/4/6
1796,150.0,2/4/6,4/25/2022,9.4,2/4/6
1797,150.0,2/4/6,4/25/2022,9.4,2/4/6
1798,150.0,2/4/6,4/25/2022,9.4,2/4/6


#### Player/Build viz and stats

In [9]:
#Exposure calc
exp_df = df2['Name_x'].value_counts()
exp_df = exp_df.reset_index()
exp_df['Count'] = exp_df['Name_x']
exp_df['%'] = round((exp_df['Count'] / df2['Team'].max()) * 100, 1)
exp_df = exp_df.drop(columns=['Name_x'])
exp_df.rename({'index' : 'Player'}, axis='columns', inplace=True)
exp_df

,Player,Count,%
0,Jahan Dotson,60,40.0
1,Rondale Moore,59,39.3
2,Treylon Burks,59,39.3
3,Drake London,57,38.0
4,Javonte Williams,51,34.0
...,...,...,...
66,Hassan Haskins,1,0.7
67,Romeo Doubs,1,0.7
68,Tyler Badie,1,0.7
69,Jerome Ford,1,0.7


In [10]:
#Player Exposure
alt.Chart(exp_df).mark_bar().encode(
    x='Count',
    y=alt.Y("Player", sort='-x'),
    tooltip=['Player','Count', '%']
).properties(height=700)

alt.Chart(...)

In [11]:
#My draft tendencies for QBs over time, by value

stripplot =  alt.Chart(df_date, width=40).mark_circle(size=8).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('Name'),
    tooltip=['Name', 'Draft Start', 'Value'],
    color=alt.Color('Value', scale=alt.Scale(scheme='orangered')),
    column=alt.Column(
        'Draft Start',
        header=alt.Header(
            labelAngle=-90,
            titleOrient='bottom',
            labelOrient='top',
            labelAlign='right',
            labelPadding=3,
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_range(
    category={'scheme' : 'dark2'}
)

stripplot

alt.Chart(...)

In [12]:
#Team Build by draft date, with cumulative ADP value

alt.Chart(df1).mark_circle(size=40).encode(
    x='Draft Start',
    y='Cumulative Val',
    color='build_x',
    tooltip=['Draft Start','Team', 'build_x', 'Cumulative Val']
).interactive()

alt.Chart(...)

In [13]:
#Breakdown of all builds
build_count = final_df['build'].value_counts()
build_count = build_count.reset_index()
build_count['Count'] = build_count['build']
build_count['%'] = round((build_count['Count'] / df2['Team'].max()) * 100, 1)
build_count = build_count.drop(columns=['build'])
build_count.rename({'index' : 'Build'}, axis='columns', inplace=True)
build_count

,Build,Count,%
0,2/3/7,59,39.3
1,2/4/6,35,23.3
2,3/3/6,27,18.0
3,3/2/7,18,12.0
4,2/2/8,10,6.7
5,3/4/5,1,0.7
6,2/4/5,1,0.7


In [14]:
#Cumulative value description
cum_val_sum = final_df['Cumulative Val'].describe()
cum_val_sum

count    151.000000
mean       6.428477
std        8.899965
min      -15.800000
25%        1.050000
50%        6.000000
75%       11.600000
max       37.700000
Name: Cumulative Val, dtype: float64

#### Player team by round search

In [15]:
#Function that returns all teams with player passed, along with the builds used with the player
def player_team_search(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])

      #Building out round by round breakdown for teams with given player
      split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
      split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)
      split_df = split_df.merge(with_player, on=['Team', 'Team'], how='right')
      split_df = split_df.drop_duplicates(subset=['Team'])
      print("Round by Round selections with " + ', '.join(players) + " on the team \n")
      while i <= rounds:
        round_count = split_df[i].value_counts().rename_axis('Player').reset_index(name='Times Selected')
        round_count = round_count.sort_index()
        print("===============================================================")
        print("                                    Round " + str(i) + " \n")
        chart = bars = alt.Chart(round_count).mark_bar().encode(
            x=alt.X('Times Selected', axis=alt.Axis(labels=True), title=""),
            y=alt.Y('Player', axis=alt.Axis(labels=True), sort='-x', title=""),
            tooltip=['Player', 'Times Selected'],
            color=alt.Color('Player', legend=None, scale=alt.Scale(scheme=alt.SchemeParams(name='darkblue')))
        ).interactive()
        chart.display(renderer='svg')
        i=i+1

      #List of teams with given player on roster
      #print("===========================================================")
      #print("Full Teams with " + ', '.join(players) + " on the roster \n")
      #print(with_player['Players'].value_counts())

#### Build Count with Given Players

In [16]:
#Build Count with given players function
def builds_with_players(players):
      base = r'^{}'
      expr = '(?=.*{})'
      base = base.format(''.join(expr.format(p) for p in players))
      #Change for number of rounds
      rounds = 20
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(base)]
      with_player = with_player.drop_duplicates(subset=['Team'])    
      print("Structural builds with " + ', '.join(players) + " on the roster")
      print(with_player['build'].value_counts())
      chart = (alt.Chart(with_player).mark_circle(size=30).encode(
      x='Draft Start',
      y='Cumulative Val',
      color='build',
      tooltip=['Draft Start','Team', 'build', 'Cumulative Val']
      ).interactive())
      chart.display(renderer='svg')
      print("\n")

#### To use while drafting

In [35]:
#Use of build and player search functions
players_to_search = ['Zach Wilson', 'Garrett Wilson']
build = builds_with_players(players_to_search)

Structural builds with Zach Wilson, Garrett Wilson on the roster
2/3/7    6
2/4/6    4
3/2/7    2
Name: build, dtype: int64


alt.Chart(...)

In [36]:
#Round by Round searching
player_team_search(players_to_search)

Round by Round selections with Zach Wilson, Garrett Wilson on the team 

                                    Round 1 



alt.Chart(...)

                                    Round 2 



alt.Chart(...)

                                    Round 3 



...

                                    Round 11 



alt.Chart(...)

                                    Round 12 



alt.Chart(...)

KeyError: ignored

In [141]:
#General ADP Guidelines. This function when given a pick will give the user the likely players availible to them in each round, given current ADP.
def adp_dist(pick):
  adp_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PPGOverReplacement/2018ADP.csv'
  adp = pd.read_csv(adp_url, index_col='ID')
  adp = adp.dropna(how='all')
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th' ]
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   
  round = 0
  rounds = 19
  while round <= rounds:
    pos = picks.iloc[round].loc[pick]
    plus = int(pos) + 5
    minus = int(pos) - 5
    temp = adp[adp['ADP'].between(minus, plus)]
    round = round + 1
    print(temp)
  



  


In [142]:
#Insert pick you have to see round by round breakdown of the players availible to you. 
adp_dist('1st')

        AGE  ADP
ID              
16771  27.0  1.3
14967  29.0  3.1
16847  30.0  3.3
17923  26.0  4.6
11056  33.0  4.7
18878  26.0  5.7
        AGE   ADP
ID               
16524  29.0  19.5
18067  27.0  20.0
12845  33.0  21.3
10974  32.0  21.5
18858  25.0  22.8
18082  27.0  24.0
16597  28.0  25.5
16906  28.0  25.6
14005  32.0  27.0
19823  25.0  27.8
15048  32.0  28.2
        AGE   ADP
ID               
18067  27.0  20.0
12845  33.0  21.3
10974  32.0  21.5
18858  25.0  22.8
18082  27.0  24.0
16597  28.0  25.5
16906  28.0  25.6
14005  32.0  27.0
19823  25.0  27.8
15048  32.0  28.2
11932  33.0  29.8
2593   38.0  30.0
        AGE   ADP
ID               
16909  28.0  44.0
14587  30.0  44.1
11611  33.0  45.5
18857  26.0  46.5
13337  32.0  47.2
11197  34.0  47.3
14865  31.0  48.3
11488  35.0  48.4
4314   44.0  50.9
16668  31.0  51.6
16568  28.0  52.0
12386  35.0  52.5
        AGE   ADP
ID               
16909  28.0  44.0
14587  30.0  44.1
11611  33.0  45.5
18857  26.0  46.5
13337  32.0  47.2


In [99]:
  picks_url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/PicksByRound.csv'
  possible_picks = ['Round','1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th' ]
  picks = pd.read_csv(picks_url, index_col='Round', names=possible_picks, encoding="utf-8-sig")
  picks = picks.dropna(how='all')
  picks = picks.iloc[1: , :]
  picks   

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th
Round,,,,,,,,,,,,
1,1,2,3,4,5,6,7,8,9,10,11,12
2,24,23,22,21,20,19,18,17,16,15,14,13
3,25,26,27,28,29,30,31,32,33,34,35,36
4,48,47,46,45,44,43,42,41,40,39,38,37
5,49,50,51,52,53,54,55,56,57,58,59,60
6,72,71,70,69,68,67,66,65,64,63,62,61
7,73,74,75,76,77,78,79,80,81,82,83,84
8,96,95,94,93,92,91,90,89,88,87,86,85
9,97,98,99,100,101,102,103,104,105,106,107,108
